# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [198]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()

#this was here to QA the bottom portion. coming from the import
#school_data_complete.groupby("type").mean() 
#I was looking for the source of the data error. 
#It appears that the sample charts were wrong and I believe mine are correct

,Student ID,reading_score,math_score,School ID,size,budget
type,,,,,,
Charter,19438.475398,83.902821,83.406183,6.964163,1717.352468,1.024543e+06
District,19650.507711,80.962485,76.987026,6.984505,4063.261195,2.611175e+06


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [187]:
#count schools
total_schools=school_data_complete["school_name"].nunique()
#count students
total_students=school_data_complete["Student ID"].count()
# go back to the school table to get the budget
budget=school_data["budget"].sum()
#calculate math scores
avg_math=school_data_complete["math_score"].mean()
#calculate reading scores
avg_read=school_data_complete["reading_score"].mean()
#count students who passed math
passed_math=school_data_complete["Student ID"].loc[school_data_complete["math_score"] >= 70].count()
#calculate percentages
mpassed_pct=passed_math/total_students * 100
#count students who passed reading
passed_read=school_data_complete["Student ID"].loc[school_data_complete["reading_score"] >= 70].count()
#calculate percentages
rpassed_pct=passed_read/total_students * 100
# do the overall percentages of who passed
opassed_pct=(passed_read+passed_math)/2/total_students*100
# well I did not read this clearly, but thought overall meant passed both reading and math.
#passed_over=school_data_complete["Student ID"].loc[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"]>=70)].count()
#opassed_pct=passed_over/total_students * 100
# note the percentage of studings passing both is 65.17

#make the summary dataframe
summary_df=pd.DataFrame({"Total Schools": [total_schools],
                         "Total Students": [total_students],
                         "Total Budget": [budget],
                         "Average Math Score": [avg_math],
                         "Average Reading Score" : [avg_read],
                         "% Passing Math":[mpassed_pct],
                         "% Passing Reading":[rpassed_pct],
                         "% Overall Passing Rate":[opassed_pct]
                    })
#clean it up
summary_df["Total Students"]=summary_df["Total Students"].map( "{:,.0f}".format)
summary_df["Total Budget"]=summary_df["Total Budget"].map( "${:,.2f}".format)
summary_df["Average Math Score"]=summary_df["Average Math Score"].map( "{:.2f}".format)
summary_df["Average Reading Score"]=summary_df["Average Reading Score"].map( "{:.2f}".format)
summary_df["% Passing Math"]=summary_df["% Passing Math"].map( "{:.2f} %".format)
summary_df["% Passing Reading"]=summary_df["% Passing Reading"].map( "{:.2f} %".format)
summary_df["% Overall Passing Rate"]=summary_df["% Overall Passing Rate"].map( "{:.2f} %".format)

#display the frame
summary_df



,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98 %,85.81 %,80.39 %


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [188]:
#create groupby
grouped_schools=school_data_complete.groupby(["school_name","type"])
#make a data frame
top_performers=grouped_schools.count()
#start building columns
top_performers["Total Students"]=top_performers["Student ID"]
#get the whole school budget as it's listed in the student, I only need one value, so I chose max
school_budget=grouped_schools.max()
#add it to the df
top_performers["Total School Budget"]=school_budget["budget"]
#drop all the other columns
top_performers=top_performers[["Total Students","Total School Budget"]]
#add per student column
top_performers["Per Student Budget"]=top_performers["Total School Budget"]/top_performers["Total Students"]
#get the average df
avg_scores=grouped_schools.mean()
#add those to the summary
top_performers["Average Math Score"]=avg_scores["math_score"]
top_performers["Average Reading Score"]=avg_scores["reading_score"]
#make a new df that contains only students who passed math
math_passers=school_data_complete[["school_name","Student ID","math_score"]].loc[school_data_complete["math_score"] >= 70]
#group them by school and count them
grouped_mp=math_passers.groupby("school_name").count()
#add them to the column at the end
top_performers["% Passing Math"]=grouped_mp["Student ID"]/top_performers["Total Students"]*100
#make a new df that contains only students who passed reading
read_passers=school_data_complete[["school_name","Student ID","reading_score"]].loc[school_data_complete["reading_score"] >= 70]
#group them by school and count them
grouped_rp=read_passers.groupby("school_name").count()
#add them to the df in a new column
top_performers["% Passing Reading"]=grouped_rp["Student ID"]/top_performers["Total Students"]*100
#do the overall calculations
top_performers["% Overall Passing Rate"]=(top_performers["% Passing Reading"]+top_performers["% Passing Math"])/2
# reset the index for type
top_performers=top_performers.reset_index(level="type")
#rename the type
top_performers=top_performers.rename(columns={"type": "School Type"})
#get rid of the schoool_name at the top
top_performers.index.names = ['']
#copying this for next one so I don't have to deal with string conversion after I add the formatting
bottom_performers=top_performers.copy()
#sort the thing
top_performers=top_performers.sort_values(by=["% Overall Passing Rate"], ascending=False)
#format the output
top_performers["Total Students"]=top_performers["Total Students"].map( "{:,.0f}".format)
top_performers["Total School Budget"]=top_performers["Total School Budget"].map( "${:,.2f}".format)
top_performers["Per Student Budget"]=top_performers["Per Student Budget"].map( "${:,.2f}".format)
top_performers["Average Math Score"]=top_performers["Average Math Score"].map( "{:.2f}".format)
top_performers["Average Reading Score"]=top_performers["Average Reading Score"].map( "{:.2f}".format)
top_performers["% Passing Math"]=top_performers["% Passing Math"].map( "{:.2f} %".format)
top_performers["% Passing Reading"]=top_performers["% Passing Reading"].map( "{:.2f} %".format)
top_performers["% Overall Passing Rate"]=top_performers["% Overall Passing Rate"].map( "{:.2f} %".format)

top_performers.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13 %,97.04 %,95.59 %
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27 %,97.31 %,95.29 %
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59 %,95.95 %,95.27 %
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39 %,97.14 %,95.27 %
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87 %,96.54 %,95.20 %


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [189]:
#This starts with the bottom performers that was copied from the last one
bottom_performers=bottom_performers.sort_values(by=["% Overall Passing Rate"], ascending=True)
#format the output
bottom_performers["Total Students"]=bottom_performers["Total Students"].map( "{:,.0f}".format)
bottom_performers["Total School Budget"]=bottom_performers["Total School Budget"].map( "${:,.2f}".format)
bottom_performers["Per Student Budget"]=bottom_performers["Per Student Budget"].map( "${:,.2f}".format)
bottom_performers["Average Math Score"]=bottom_performers["Average Math Score"].map( "{:.2f}".format)
bottom_performers["Average Reading Score"]=bottom_performers["Average Reading Score"].map( "{:.2f}".format)
bottom_performers["% Passing Math"]=bottom_performers["% Passing Math"].map( "{:.2f} %".format)
bottom_performers["% Passing Reading"]=bottom_performers["% Passing Reading"].map( "{:.2f} %".format)
bottom_performers["% Overall Passing Rate"]=bottom_performers["% Overall Passing Rate"].map( "{:.2f} %".format)
bottom_performers.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37 %,80.22 %,73.29 %
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99 %,80.74 %,73.36 %
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68 %,81.32 %,73.50 %
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06 %,81.22 %,73.64 %
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31 %,79.30 %,73.80 %


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [190]:
#This creates data frames for all of the grades and then groups them by schools
ninth_df=school_data_complete.loc[school_data_complete["grade"]=="9th"]
ninth_groupby=ninth_df.groupby("school_name").mean()
tenth_df=school_data_complete.loc[school_data_complete["grade"]=="10th"]
tenth_groupby=tenth_df.groupby("school_name").mean()
eleventh_df=school_data_complete.loc[school_data_complete["grade"]=="11th"]
eleventh_groupby=eleventh_df.groupby("school_name").mean()
twelfth_df=school_data_complete.loc[school_data_complete["grade"]=="12th"]
twelfth_groupby=twelfth_df.groupby("school_name").mean()

#this builds out the display dataframe
math_display=ninth_groupby
math_display["9th"]=math_display["math_score"]
math_display["10th"]=tenth_groupby["math_score"]
math_display["11th"]=eleventh_groupby["math_score"]
math_display["12th"]=twelfth_groupby["math_score"]
math_display=math_display[["9th","10th","11th","12th"]]
math_display.index.names = ['']


math_display

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [191]:
#beginning from the same grade groups as the previous frame, this makes the table for the readings
#this builds out the display dataframe
reading_display=ninth_groupby
reading_display["9th"]=reading_display["reading_score"]
reading_display["10th"]=tenth_groupby["reading_score"]
reading_display["11th"]=eleventh_groupby["reading_score"]
reading_display["12th"]=twelfth_groupby["reading_score"]
reading_display=reading_display[["9th","10th","11th","12th"]]
reading_display.index.names = ['']


reading_display

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [16]:
#keep this one

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [192]:
# I think I need to make a column here to join onto the main df that adds the spending per student
# I'm going to go ahead and add the school size on that too
# It's at this point that I realized that school size is in the table to start with and I've made this harder than it needs to be in previous blocks
# It's not worth the work to fix that as the answer is correct
# I do need to ad the spending per student column, though
school_data_complete["spend per kid"]=school_data_complete["budget"]/school_data_complete["size"]

# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#add the bins to the data
school_data_complete["Spending Ranges (Per Student)"]=pd.cut(school_data_complete["spend per kid"], spending_bins, labels=group_names)
grouped_spending=school_data_complete.groupby("Spending Ranges (Per Student)").mean()

# Add new columns instead of renaming them
grouped_spending["Average Math Score"]=grouped_spending["math_score"]
grouped_spending["Average Reading Score"]=grouped_spending["reading_score"]
# Drop all the columns that aren't going to be in final
grouped_spending=grouped_spending[["Average Math Score","Average Reading Score"]]
#group math passers
gmp=school_data_complete[["Spending Ranges (Per Student)","Student ID","math_score"]].loc[school_data_complete["math_score"] >= 70]
gmpg=gmp.groupby("Spending Ranges (Per Student)").count()
#now that we have a count of the passers, we need to find the number of students in the group
students_in_group=school_data_complete.groupby("Spending Ranges (Per Student)").count()
#use it to calculate the percentages
grouped_spending["% Passing Math"]=gmpg["Student ID"]/students_in_group["Student ID"] *100
#group reading passers
grp=school_data_complete[["Spending Ranges (Per Student)","Student ID","reading_score"]].loc[school_data_complete["reading_score"] >= 70]
grpg=grp.groupby("Spending Ranges (Per Student)").count()
#calculate percentages
grouped_spending["% Passing Reading"]=grpg["Student ID"]/students_in_group["Student ID"]*100
#do the average of the two for the final
grouped_spending["% Overall Passing Rate"]=(grouped_spending["% Passing Reading"]+grouped_spending["% Passing Math"])/2


grouped_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [193]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# add the bin column
school_data_complete["School Size"]=pd.cut(school_data_complete["size"], size_bins, labels=group_names)

#group by bins to start the table
size_group=school_data_complete.groupby("School Size").mean()
#rename some things
size_group["Average Math Score"]=size_group["math_score"]
size_group["Average Reading Score"]=size_group["reading_score"]
#drop some columns
size_group=size_group[["Average Math Score","Average Reading Score"]]
#size group math passers
sgmp=school_data_complete[["School Size","Student ID","math_score"]].loc[school_data_complete["math_score"] >= 70]
sgmpg=sgmp.groupby("School Size").count()
#now that we have a count of the passers, we need to find the number of students in the group
size_students_in_group=school_data_complete.groupby("School Size").count()
#use it to calculate the percentages
size_group["% Passing Math"]=sgmpg["Student ID"]/size_students_in_group["Student ID"] *100
#size group reading passers
sgrp=school_data_complete[["School Size","Student ID","reading_score"]].loc[school_data_complete["reading_score"] >= 70]
sgrpg=sgrp.groupby("School Size").count()
#calculate the percentage
size_group["% Passing Reading"]=sgrpg["Student ID"]/size_students_in_group["Student ID"]*100
#do overall percentages
size_group["% Overall Passing Rate"]=(size_group["% Passing Reading"]+size_group["% Passing Math"])/2


size_group

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [194]:
type_group=school_data_complete.groupby("type").mean()

#rename some things
type_group["Average Math Score"]=type_group["math_score"]
type_group["Average Reading Score"]=type_group["reading_score"]
#drop some columns
type_group=type_group[["Average Math Score","Average Reading Score"]]
#size group math passers
type_mpassers=school_data_complete[["type","Student ID","math_score"]].loc[school_data_complete["math_score"] >= 70]
tmpg=type_mpassers.groupby("type").count()
#now that we have a count of the passers, we need to find the number of students in the group
type_students_in_group=school_data_complete.groupby("type").count()
#use it to calculate the percentages
type_group["% Passing Math"]=tmpg["Student ID"]/type_students_in_group["Student ID"] *100
#size group reading passers
type_rpassers=school_data_complete[["type","Student ID","reading_score"]].loc[school_data_complete["reading_score"] >= 70]
trpg=type_rpassers.groupby("type").count()
#calculate the percentage
type_group["% Passing Reading"]=trpg["Student ID"]/type_students_in_group["Student ID"]*100
#do overall percentages
type_group["% Overall Passing Rate"]=(type_group["% Passing Reading"]+type_group["% Passing Math"])/2


type_group



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818


In [20]:
#don't type here as I want the graph for later

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [195]:
school_data_complete.groupby("type").mean()

,Student ID,reading_score,math_score,School ID,size,budget,spend per kid
type,,,,,,,
Charter,19438.475398,83.902821,83.406183,6.964163,1717.352468,1.024543e+06,598.778498
District,19650.507711,80.962485,76.987026,6.984505,4063.261195,2.611175e+06,643.087300


In [196]:
school_data_complete.info()
#

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39170 entries, 0 to 39169
Data columns (total 14 columns):
Student ID                       39170 non-null int64
student_name                     39170 non-null object
gender                           39170 non-null object
grade                            39170 non-null object
school_name                      39170 non-null object
reading_score                    39170 non-null int64
math_score                       39170 non-null int64
School ID                        39170 non-null int64
type                             39170 non-null object
size                             39170 non-null int64
budget                           39170 non-null int64
spend per kid                    39170 non-null float64
Spending Ranges (Per Student)    39170 non-null category
School Size                      39170 non-null category
dtypes: category(2), float64(1), int64(6), object(5)
memory usage: 4.0+ MB
